## Network Intrusion Detection

**NOTE: This notebook is worth 60% of the grade of project 2.**

In the last four exercises, we use the KDD data set to practice how to create and operate on RDDs. In this project, we will focus on the main purpose of the KDD data set, which is a sample data set for **the Ihird International Knowledge Discovery and Data Mining Tools Competition (KDD cup 99)**.

The KDD cup 99 page describes the motivation of the competition as follows:
> Software to detect network intrusions protects a computer network from unauthorized users, including perhaps insiders.  The intrusion detector learning task is to build a predictive model (i.e. a classifier) capable of distinguishing between "bad" connections, called intrusions or attacks, and "good" normal connections.

The classification of the KDD cup 99 mainly targets four types of attacks:
- **DOS**: denial-of-service, e.g. syn flood;
- **R2L**: unauthorized access from a remote machine, e.g. guessing password;
- **U2R**:  unauthorized access to local superuser (root) privileges, e.g., various "buffer overflow" attacks;
- **probing**: surveillance and other probing, e.g., port scanning.

Although the main target of the KDD cup 99 is **knowledge Discovery** and **Data Mining**, this project will not involve any training and prediction. We will simply use pyspark to explore some well-known rules to extract some useful information from the KDD 99 data set.

### Reading the KDD 99 Data Set

In this notebook we will use the reduced dataset (1 percent) provided for the KDD Cup 1999, containing nearly half million network interactions. The file is provided as a *Gzip* file in the local directory.  

In [ ]:
import os
data_file = "file://" + os.getcwd() + "/../kddcup.data_1_percent.gz"
raw_data = sc.textFile(data_file)

Now we have created an RDD from the data set. The RDD data sets are structured as the csv format, with each row (i.e., each line) containing the fields of a **network interaction**. The fields in the same row are separated with commas (,). According to <http://kdd.ics.uci.edu/databases/kddcup99/task.html>, each row in the RDD data set contains the following four type of fields:

- Basic features of individual TCP connections:
 
|feature name  |description                                                  | type       |
|--------------|-------------------------------------------------------------|------------|
|duration 	   |length (number of seconds) of the connection                 | continuous |
|protocol_type |type of the protocol, e.g. tcp, udp, etc.                    | discrete   |
|service 	   |network service on the destination, e.g., http, telnet, etc. | discrete   |
|src_bytes 	   |number of data bytes from source to destination 	         | continuous |
|dst_bytes 	   |number of data bytes from destination to source 	         | continuous |
|flag 	       |normal or error status of the connection 	                 | discrete   | 
|land 	       |1 if connection is from/to the same host/port; 0 otherwise 	 | discrete   |
|wrong_fragment|number of "wrong" fragments 	                             | continuous |
|urgent 	   |number of urgent packets 	                                 | continuous |

- Content features within a connection suggested by domain knowledge:
 
|feature name      |description                                                  | type       |
|------------------|-------------------------------------------------------------|------------|
|hot 	           |number of "hot" indicators	                                 |continuous  |
|num_failed_logins |number of failed login attempts 	                         |continuous  |
|logged_in         |1 if successfully logged in; 0 otherwise 	                 |discrete    |
|num_compromised   |number of "compromised" conditions 	                         |continuous  |
|root_shell        |1 if root shell is obtained; 0 otherwise 	                 |discrete    |
|su_attempted      |1 if `su root` command attempted; 0 otherwise 	             |discrete    |
|num_root          |number of "root" accesses 	                                 |continuous  |
|num_file_creations|number of file creation operations 	                         |continuous  |
|num_shells        |number of shell prompts 	                                 |continuous  |
|num_access_files  |number of operations on access control files 	             |continuous  |
|num_outbound_cmds |number of outbound commands in an ftp session 	             |continuous  |
|is_hot_login      |1 if the login belongs to the "hot" list; 0 otherwise        |discrete    |
|is_guest_login    |1 if the login is a "guest" login; 0 otherwise               |discrete    |

- Traffic features computed using a two-second time window:
 
|feature name  |description                                                  | type       |
|--------------|-------------------------------------------------------------|------------|
|count         |number of connections to the same host as the current connection in the past two seconds|continuous|
|-             |Note: The following  features refer to these same-host connections.|      |	
|serror_rate   |% of connections that have "SYN" errors 	                 |continuous  |
|rerror_rate   |% of connections that have "REJ" errors 	                 |continuous  |
|same_srv_rate |% of connections to the same service 	                     |continuous  |
|diff_srv_rate  |% of connections to different services                      |continuous  |
|srv_count      |number of connections to the same service as the current connection in the past two seconds|continuous |
|-              |Note: The following features refer to these same-service connections.|    |	
|srv_serror_rate|% of connections that have "SYN" errors                     |continuous  |
|srv_rerror_rate|% of connections that have "REJ" errors                     |continuous  |
|srv_diff_host_rate|% of connections to different hosts                      |continuous  |

- **Tags**: classification of network interactions. This field shows the classification of the attack factor for each interactions. Possible values of tags are: back, buffer_overflow, ftp_write, guess_passwd, imap, ipsweep, land, loadmodule, multihop, neptune, nmap, normal, perl, phf, pod, portsweep, rootkit, satan, smurf, spy, teardrop, warezclient, warezmaster.

Now, based on the data set, please list the top 10 attack factors (i.e., tags that are **not** "normal") and print in a readable format. Note: You must use RDDs to implement your algorithm. Processing the data locally without RDDs is not acceptable.

In [ ]:
t0 = time()

# First RDD:  normal_interactions = interaction with tag = "normal."

# Second RDD: attack_interactions = interaction with tag != "normal."

# third RDD:  attack_tag_counts = interaction count of each tag which is not "normal".

# fourth RDD: attack_tag_counts_sorted = sorted list of tags with their interaction counts.

# result:     attack_tag_counts_top10 = top 10 attack tags and their interaction counts.

tt = time() - t0

for (tag, count) in attack_tag_counts_top10:
    print(tag + ": counts = " + str(count))
    
print("Evaluation has taken {} secs".format(round(tt,3)))

### Distributed Denial of Service (DDOS)

The KDD 99 data set has defined six primary types of DOS attacks: **back**, **land**, **neptune**, **pod**, **smurf**, **teardrop**. Without further details of these DOS attacks, can you identified the attacks which are **most distributed**, as well as the attacks which are **most correlated with SYN errors**?

First, filter the network interactions in the raw data to contain only these six types of DOS attacks:

In [ ]:
t0 = time()

# Filter the network interactions to include only DOS attacks
# dos_attack_data = ...

# Calculate the count of DOS interactions for each tag

tt = time() - t0

for (tag, count) in dos_attack_counts:
    print(tag + ": counts = " + str(count))

print("Filtering DOS attacks has taken {} secs".format(round(tt,3)))

Then, sort the DOS attacks to show, from the highest to the lowest, **the average numbers of connections** within the last 2 seconds (in regards to each host) involved in each DOS attack. Hint: use the mean values of the field `count` to determine the **most distributed** DOS attacks.

In [ ]:
t0 = time()

# Calculate the mean values of distribution degrees (# of connections involved with the interaction)
# ...
# sorted_dos_attack_degrees = ...

tt = time() - t0

for (tag, degree) in sorted_dos_attack_degrees:
    print(tag + ": degree = " + str(degree))

print("Sorting DOS attacks by degree of distribution has taken {} secs".format(round(tt,3)))

Finally, sort the DOS attacks show, from the highest to the lowest, **the average numbers of SYN errors** within the last 2 seconds (in regards to each host) involved in each DOS attack. Hint: approximate the numbers of SYN errors by multiplying `count` with `serror_rate`.

In [ ]:
t0 = time()

# Calculate the average numbers of connections with SYN errors (# of connections involved with the interaction)
# ...
# sorted_dos_attack_syn_error_counts = ...

tt = time() - t0

for (tag, count) in sorted_dos_attack_syn_error_counts:
    print(tag + ": degree = " + str(count))

print("Sorting DOS attacks by correlation with SYN flooding has taken {} secs".format(round(tt,3)))

### Brute-force Login Attacks

A brute-force login attack relies on the attacker continuously re-attempting failed login to a host until being able to log into the host and eventually to perform `su` to become the root user. A host often has defense mechanisms such as ASLR (Address Space Layout Randomization) to reduce the probability for an attack to successfully perform `su` following successful login.

Now we will calculate the **the average number of failed login attempt** before successful login (i.e., `logged_in == 1`), and **the average number of failed login attempt** before successful `su` to gain the root shell (i.e., `root_shell == 1`).

In [ ]:
t0 = time()

# Calculate the average numbers of failed login attempt before successful login
# average_attempt_before_login = ...

# Calculate the average numbers of failed login attempt before successful `su` to gain the root shell
# average_attempt_before_su = ...

tt = time() - t0


print("Average number of attempts before successful login: {}".format(average_attempt_before_login))
print("Average number of attempts before successful su: {}".format(average_attempt_before_su))

print("Calculating the difficulty of brute-force login attacks has taken {} secs".format(round(tt,3)))

### You are done!

In [ ]:
# If you have finished this notebook and are ready to submit, please uncomment and
# execute the following code to let the TA know.
# print "Ready for grading."